In [1]:
import glob

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
awkward_repos = np.array(
    [
        202413762,  # scikit-hep/awkward-1.0
        137079949,  # scikit-hep/awkward-array
        202413762,  # scikit-hep/awkward
        137079949,  # scikit-hep/awkward-0.x
        398059046,  # dask-contrib/dask-awkward
        511652265,  # intake/awkward-pandas
    ]
)

dfs = []
for filename in sorted(glob.glob("/home/jpivarski/storage/data/GHArchive/GHArchive-202[2-3].h5")):
    print(filename)
    with h5py.File(filename, mode="r") as file:
        is_awkward = np.isin(file["repo_id"], awkward_repos)
        dfs.append(
            pd.DataFrame(
                {
                    "timestamp": file["timestamp"][is_awkward],
                    "actor_id": file["actor_id"][is_awkward],
                    "event_type_id": file["event_type_id"][is_awkward],
                }
            )
        )

/home/jpivarski/storage/data/GHArchive/GHArchive-2022.h5
/home/jpivarski/storage/data/GHArchive/GHArchive-2023.h5


In [13]:
df = pd.concat(dfs, ignore_index=True)

In [14]:
df.sort_values("timestamp", inplace=True)

In [15]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

In [16]:
df.set_index("timestamp", inplace=True)

In [17]:
fewer_event_types = {
     1: "comment",
     2: "",
     3: "",
     4: "",
     5: "star",
     6: "contribute",
     7: "star",
     8: "",
     9: "",
    10: "comment",
    11: "contribute",
    12: "",
    13: "",
    14: "contribute",
    15: "comment",
    16: "comment",
    17: "contribute",
    18: "",
    19: "",
    20: "star",
}

In [18]:
df["is_comment"] = df["event_type_id"].map(lambda x: fewer_event_types[x] == "comment")
df["is_contribute"] = df["event_type_id"].map(lambda x: fewer_event_types[x] == "contribute")

In [37]:
np.sum((df.index > "2021-09-01") & df["is_comment"])

5909

In [38]:
np.sum((df.index > "2021-09-01") & df["is_contribute"])

5663

In [39]:
df[(df.index > "2021-09-01") & (df["is_comment"] | df["is_contribute"])].reset_index()["actor_id"].unique().shape

(79,)